In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [18]:
import numpy as np
import cv2

# DÜZELTME 1: 'np.ndarry' -> 'np.ndarray' olarak düzeltildi (Senin hatan buydu)
def cropped_black_borders(img: np.ndarray, thr=12, pad=0):
    # thr parametresi thresh değerini temsil ediyor. Arkaplan siyah renginde olduğu için 0'a çok yakın bir değer verdik
    # pad parametresi paddingi temsil ediyor. Görüntüyü arkaplandan ayırt ettikten sonra sol sağ üst ve alt
    # raflarında boşluk bırakılıp bırakılmayacağına karar veriyor. Biz varsayılan olarak 0 değeri girdiğimiz için 4 taraftan da
    # tamamen ayırt edilmiş/kesilmiş tam bir görüntü elde etmek istediğimizi belirtiyoruz.

    # ilk olarak eğer görüntü renkli bir görüntü ise yani 3 kanallı ise bunun kontrolünü sağlıyoruz.
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    # görüntü tek kanallı hale geldikten sonra ilk olarak arka plandaki siyah piksellerden ayırt etmek için 1 maske çıkarıyoruz.
    mask = (gray > thr).astype(np.uint8) * 255
    kernel = np.ones((5, 5), dtype=np.uint8)

    # yukarıda elimizde bi harita(mask) ve bir matris(filtreleme matrisi) var artık
    # Morfolojik kapanış (Close) işlemi, maske üzerindeki küçük siyah delikleri ve gürültüleri kapatır.
    thresh = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # DÜZELTME 2: Konturları, gürültülü 'mask' yerine temizlenmiş 'thresh' üzerinde bulmalıyız.
    contour, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # DÜZELTME 3: Eğer hiç kontur bulunamazsa (görüntü tamamen siyahsa) hata almamak için kontrol ekle.
    if not contour:
        print("Uyarı: Siyah olmayan alan bulunamadı. Orijinal görüntü döndürülüyor.")
        return img  # Orijinal görüntüyü döndür

    # En büyük alanı kaplayan konturu seç
    c = max(contour, key=cv2.contourArea)
    
    # seçtiğimiz en büyük konturu(0,0) noktasından x,y tüm sutun ve tum satırları elde edecek şekilde bir dikdörtgene dönüştürüyoruz.
    x, y, w, h = cv2.boundingRect(c)

    # kırpma işlemi esnasında görüntünün sınırlarını aşılmaması için görüntünün mevcut yükselkik ve genişlik bilgisinide elde etmemiz gerekiyor
    # bix görüntüyü orijinal görüntü içerisinden crop yaparken sağ sol üst alt bölgelerden herhangi bir kayma veya aşınma olmasın istiyoruz.

    # DÜZELTME 4: 'img.shape=[:2]' hatalı syntax. Doğrusu 'img.shape[:2]' olmalı.
    H, W = img.shape[:2]
    
    # görüntünün yükselkiği ve genişliğini elde ettikten sonra cropper işlemine geçiyoruz
    # köşeleri tespit ediyoruz

    x1 = max(0, x - pad)
    
    # DÜZELTME 5: 'y1' için 'x - pad' değil, 'y - pad' kullanılmalı (kopyala-yapıştır hatası).
    y1 = max(0, y - pad)
    
    x2 = min(W, x + w + pad)
    y2 = min(H, y + h + pad)

    return img[y1:y2, x1:x2]

# --- ÖRNEK KULLANIM ---
# (Bu kısmı yorumdan çıkarıp test edebilirsin)
#
# my_bgr_img = cv2.imread("yeni1.png")
# if my_bgr_img is not None:
#     my_cropped_img = cropped_black_borders(my_bgr_img)
#
#     cv2.imshow("Original Image", my_bgr_img)
#     cv2.imshow("Cropped Image", my_cropped_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# else:
#     print("Hata: Görüntü yüklenemedi. 'yeni1.png' dosya adını kontrol et.")









        

In [24]:
my_bgr_img = cv2.imread("yeni1.png")

if my_bgr_img is not None:
    my_cropped_img = cropped_black_borders(my_bgr_img)

    cv2.imshow("cropped image", my_cropped_img)
    
  
    cv2.waitKey(0) 
    
    cv2.destroyAllWindows()
